In [1]:
import pandas as pd


In [3]:
# Load the uploaded option chain CSV
file_path = "/Users/arindamdas/Documents/Study Resource/Quant Finance/QuantProjects/quantfolio/quant-projects/option_pricing/nifty_option_chain.csv"
df = pd.read_csv(file_path)
# Display basic info and first few rows
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   type    1378 non-null   object 
 1   expiry  1378 non-null   object 
 2   strike  1378 non-null   int64  
 3   IV      1378 non-null   float64
 4   LTP     1378 non-null   float64
 5   Volume  1378 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 64.7+ KB


(None,
    type       expiry  strike     IV      LTP  Volume
 0  Call  03-Jul-2025   22800   0.00  2720.00       0
 1   Put  03-Jul-2025   22800  33.56     0.90  503748
 2  Call  03-Jul-2025   22850   0.00  2205.45       0
 3   Put  03-Jul-2025   22850  32.81     0.85  139933
 4  Call  03-Jul-2025   22900   0.00  2629.25       0)

In [4]:
# Convert strike to int (if not already), and filter for 03-Jul-2025 expiry
df["strike"] = pd.to_numeric(df["strike"], errors='coerce')
df["expiry"] = df["expiry"].astype(str)

# Define the spot price and strike range
spot_price = 25637
atm_strike = round(spot_price / 100) * 100  # nearest 100 = 25600
strike_range = list(range(atm_strike - 1000, atm_strike + 1001, 100))
# Filter for expiry = 03-Jul-2025 and required strikes
filtered_df = df[
    (df["expiry"] == "03-Jul-2025") &
    (df["strike"].isin(strike_range))
].copy()
# Sort by strike and type for clarity
filtered_df.sort_values(by=["strike", "type"], inplace=True)

filtered_df.reset_index(drop=True, inplace=True)
filtered_df.head(10)


,type,expiry,strike,IV,LTP,Volume
0,Call,03-Jul-2025,24600,0.00,1061.05,327
1,Put,03-Jul-2025,24600,17.65,6.40,239541
2,Call,03-Jul-2025,24700,0.00,966.50,582
3,Put,03-Jul-2025,24700,16.83,7.80,294764
4,Call,03-Jul-2025,24800,0.00,869.10,1482
5,Put,03-Jul-2025,24800,16.01,9.75,383746
6,Call,03-Jul-2025,24900,0.00,772.00,1890
7,Put,03-Jul-2025,24900,15.24,12.25,309912
8,Call,03-Jul-2025,25000,0.00,671.50,25013
9,Put,03-Jul-2025,25000,14.48,15.85,825030


In [6]:
from datetime import datetime
from scipy.stats import norm
import numpy as np

# Get ATM option rows
atm_strike = 25600
atm_options = filtered_df[filtered_df["strike"] == atm_strike]

# Market LTPs and IVs for Call and Put
call_row = atm_options[atm_options["type"] == "Call"].iloc[0]
put_row = atm_options[atm_options["type"] == "Put"].iloc[0]



In [7]:
S = spot_price
K = atm_strike
IV_call = call_row["IV"] / 100 if call_row["IV"] > 0 else 0.20  # fallback IV = 20%
IV_put = put_row["IV"] / 100 if put_row["IV"] > 0 else 0.20
LTP_call = call_row["LTP"]
LTP_put = put_row["LTP"]
r = 0.07
today = datetime(2025, 6, 29)
expiry = datetime.strptime("03-Jul-2025", "%d-%b-%Y")
T = (expiry - today).days / 365  # time to expiry in years

In [8]:
# Given values
S = 25637                # Spot price
K = 25600                # ATM Strike
r = 0.07                 # Risk-free rate
IV_call = 9.8 / 100      # From your data
IV_put = 10.87 / 100     # From your data
LTP_call = 170.0         # Market LTP (Call)
LTP_put = 107.4          # Market LTP (Put)
T = (datetime(2025, 7, 3) - datetime(2025, 6, 29)).days / 365  # Time to expiry in years


In [9]:

# Black-Scholes Function
def bs_price(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Calculate theoretical prices
bs_call = bs_price(S, K, T, r, IV_call, option_type="call")
bs_put = bs_price(S, K, T, r, IV_put, option_type="put")

In [10]:

print(f"📈 Call Option (ATM 25600): Market LTP = {LTP_call}, BS Price = {bs_call:.2f}")
print(f"🧾 Status: {'Overpriced' if LTP_call > bs_call else 'Underpriced'}")

print(f"\n📉 Put Option (ATM 25600): Market LTP = {LTP_put}, BS Price = {bs_put:.2f}")
print(f"🧾 Status: {'Overpriced' if LTP_put > bs_put else 'Underpriced'}")

📈 Call Option (ATM 25600): Market LTP = 170.0, BS Price = 135.55
🧾 Status: Overpriced

📉 Put Option (ATM 25600): Market LTP = 107.4, BS Price = 90.13
🧾 Status: Overpriced
